# Healthy Plate Builder

Our BILD 62 final project aims to promote healthier, yet tasty, eating habits. The USA has many unhealthy food options and has oversaturated our daily meals with fast and overprocessed foods. Those options are not nutritional and beneficial for a balanced diet, so using our program would help guide people in a better direction regarding choosing what to eat. It creates interactive solutions to promote a better and mindful diet.

## About this project
By utilizing data from the USDA food database we would like to offer an interactive plate builder where users can build their plate with a selection of foods that are nutritious (fruits, veggies, protein, dairy, carbohydrates). Once a user has finished building their plate we hope to provide a nutrional analysis of the plate with details on balanced the meal is and suggestions for meal improvements.

In [ ]:
# this is where we will have our final project 

### Part 1: Importing Data

This part of our project we aim to complete a couple of things: 1.Succesfully import the data we'd like to use , 2. Clean our data, 3. Have our data ready for use in the following parts.


In [1]:
import os
os.getcwd()


'/Users/keeks/Bild62'

### Part 2: Design our Interactive Aspects

This part of our project we aim to sort out how we would like for user input to work

### Part 3: Nutritional Analysis

This part of our project we aim to do the necessary work that will allow for user input to be translated into nutritional details.

### Part 4: Visualization

This part of our project we aim to neatly display the results of Parts 1 and 2 to our users. 

### Part 5: Healthy Plate Builder in Action

This part of our project is where we'd like for our users to be able to use our interactive plate builder . 